# Start-to-Finish Example: Numerical Solution of the BSSN Equations, in Curvilinear Coordinates

## This module solves Einstein's equations for a merging black hole system in *spherical coordinates*

### NRPy+ Source Code for this module: [BSSN/BSSN_RHSs.py](../edit/BSSN/BSSN_RHSs.py); [Brill-Lindquist initial data module](../edit/Tutorial-EinsteinsEquations__Brill-Lindquist_Initial_Data.ipynb)

We first use NRPy+ to generate initial data for the scalar wave equation, and then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. <font color='green'>Set gridfunction values to initial data (*Part 1* below).</font>
1. Evolve the system forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. <font color='green'>Evaluate BSSN RHS expressions.</font>
    1. Apply singular, curvilinear coordinate boundary conditions [*a la* the SENR/NRPy+ paper](https://arxiv.org/abs/1712.07658)
    1. <font color='green'>Apply constraints on conformal 3-metric: $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$</font>
1. At the end of each iteration in time, output the <font color='green'>Hamiltonian constraint violation</font>.
1. Repeat above steps at two numerical resolutions to confirm convergence to zero.

## Part 1: Setting up the initial data

### Brill-Lindquist initial data ([Brill & Lindquist, Phys. Rev. 131, 471, 1963](https://journals.aps.org/pr/abstract/10.1103/PhysRev.131.471); see also Eq. 1 of [Brandt & Brügmann, arXiv:gr-qc/9711015v1](https://arxiv.org/pdf/gr-qc/9711015v1.pdf)) may be written in terms of the BSSN conformal factor and ADM extrinsic curvature as

$$\psi = e^{\phi} = 1 + \sum_{i=1}^N \frac{m_{(i)}}{2 \left|\vec{r}_{(i)} - \vec{r}\right|};\quad K_{ij}=0.$$

These data consist of $N$ nonspinning black holes initially at rest. This module restricts to the case of two such black holes, positioned along either the $x$ or $z$ axis. Here, we implement $N=2$.

**Inputs for $\psi$**:
* The position and (bare) mass of black hole 1: $\left(x_{(1)},y_{(1)},z_{(1)}\right)$ and $m_{(1)}$, respectively
* The position and (bare) mass of black hole 2: $\left(x_{(2)},y_{(2)},z_{(2)}\right)$ and $m_{(2)}$, respectively

**Additional variables needed for spacetime evolution**:
* Desired coordinate system
* Desired initial lapse $\alpha$ and shift $\beta^i$

**Transformation to curvilinear coordinates**:
* Once the above variables have been set in Cartesian coordinates, we will apply the appropriate coordinate transformations and tensor rescalings ([described in the BSSN NRPy+ tutorial module](Tutorial-BSSNCurvilinear.ipynb))

In [1]:
# Step P0: Load needed modules
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import reference_metric as rfm
import BSSN.BSSN_RHSs

# Step P1: Set up reference metric
# Step P1a: Choose SinhSpherical coordinates
CoordSystem = "SinhSpherical"
par.set_parval_from_str("reference_metric::CoordSystem",CoordSystem)
# Step P1b: Set up needed reference metric quantities
rfm.reference_metric()

thismodule = "Brill-Lindquist"
BH1_posn_x,BH1_posn_y,BH1_posn_z = par.Cparameters("REAL", thismodule, ["BH1_posn_x","BH1_posn_y","BH1_posn_z"])
BH1_mass = par.Cparameters("REAL", thismodule, ["BH1_mass"])
BH2_posn_x,BH2_posn_y,BH2_posn_z = par.Cparameters("REAL", thismodule, ["BH2_posn_x","BH2_posn_y","BH2_posn_z"])
BH2_mass = par.Cparameters("REAL", thismodule, ["BH2_mass"])

# Step 0: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

#x = rfm.xxCart[0]
#y = rfm.xxCart[1]
#z = rfm.xxCart[2]

Cartxyz = ixp.declarerank1("Cartxyz")

# Step 1: Set psi, the conformal factor:
psi = sp.sympify(1)
psi += BH1_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH1_posn_x)**2 + (Cartxyz[1]-BH1_posn_y)**2 + (Cartxyz[2]-BH1_posn_z)**2) )
psi += BH2_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH2_posn_x)**2 + (Cartxyz[1]-BH2_posn_y)**2 + (Cartxyz[2]-BH2_posn_z)**2) )

# Step 2: Set all needed ADM variables in Cartesian coordinates
gammaCartDD = ixp.zerorank2()
KCartDD     = ixp.zerorank2() # K_{ij} = 0 for these initial data
for i in range(DIM):
    gammaCartDD[i][i] = psi**4

alpha = 1/psi**2
betaCartU = ixp.zerorank1() # We generally choose \beta^i = 0 for these initial data

Next we convert these ADM variables 

$$\left\{\gamma_{ij}, K_{ij}, \alpha, \beta^i\right\}$$

to the BSSN variables

$$\left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\},$$ 

to the rescaled variables 

$$\left\{h_{i j},a_{i j},\phi, K, \lambda^{i}, \alpha, \mathcal{V}^i, \mathcal{B}^i\right\},$$ 

in this case assuming that all components of the initial shift and its time derivative are zero: $\beta^i=\mathcal{V}^i=\mathcal{B}^i=0$.

First we have (Eqs. 1 and 2 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)):
$$
\bar{\gamma}_{i j} = \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/3} \gamma_{ij}.
$$
Brill-Linquist initial data are conformally flat, meaning that $\bar{\gamma}=\hat{\gamma}=1$ in Cartesian coordinates. So for these initial data in Cartesian coordinates we have
$$
\bar{\gamma}_{i j} = \gamma^{1/3} \gamma_{ij}.
$$

In [2]:
# Step 3a: Convert ADM $\gamma_{ij}$ to BSSN $\bar{\gamma}_{ij}$
gammaCartUU, gammaCartDET = ixp.symm_matrix_inverter3x3(gammaCartDD)
gammaCartbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        gammaCartbarDD[i][j] = gammaCartDET**(sp.Rational(1,3))*gammaCartDD[i][j]

Second, we convert $K_{ij}$ to $\bar{A}_{ij}$ and $K$, where (Eq. 3 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)):

\begin{align}
K &= \gamma^{ij} K_{ij} \\
\bar{A}_{ij} &= \left(\frac{\gamma}{\bar{\gamma}}\right)^{1/3} \left(K_{ij} - \frac{1}{3} \gamma_{ij} K \right)\\
&= \gamma^{1/3} \left(K_{ij} - \frac{1}{3} \gamma_{ij} K \right)
\end{align}

In [3]:
# Step 3b: Convert ADM $K_{ij}$ to $\bar{A}_{ij}$ and $K$, 
#          where (Eq. 3 of Baumgarte et al.: https://arxiv.org/pdf/1211.6632.pdf)
KCart = sp.sympify(0)
for i in range(DIM):
    for j in range(DIM):
        KCart += gammaCartUU[i][j]*KCartDD[i][j]

ACartbarDD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        ACartbarDD[i][j] = gammaCartDET**(sp.Rational(1,3))*(KCartDD[i][j] - sp.Rational(1,3)*gammaCartDD[i][j]*KCart)

Third, we define $\bar{\Lambda}^i$ (Eqs. 4 and 5 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf)):

$$
\bar{\Lambda}^i = \bar{\gamma}^{jk}\left(\bar{\Gamma}^i_{jk} - \hat{\Gamma}^i_{jk}\right).
$$

In Cartesian coordinates, $\hat{\Gamma}^i_{jk}=0$, so we get in Cartesian coordinates
\begin{align}
\bar{\Lambda}^i &= \bar{\gamma}^{jk} \bar{\Gamma}^i_{jk} \\
&= \bar{\gamma}^{jk} \frac{1}{2} \bar{\gamma}^{il} \left(
\bar{\gamma}_{lj,k} + \bar{\gamma}_{lk,j} - \bar{\gamma}_{jk,l} \right)
\end{align}

In [4]:
# Step 3c: Define $\bar{\Lambda}^i$ from Eqs. 4 and 5 of Baumgarte et al.: https://arxiv.org/pdf/1211.6632.pdf
LambdaCartU = ixp.zerorank1()

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            for l in range(DIM):
                LambdaCartU[i] += sp.Rational(1,2)*gammaCartUU[j][k]*gammaCartUU[i][l]* \
                                    ( sp.diff(gammaCartUU[l][j],Cartxyz[k]) + 
                                      sp.diff(gammaCartUU[l][k],Cartxyz[j]) -
                                      sp.diff(gammaCartUU[j][k],Cartxyz[l]) )

All BSSN tensors and vectors are in Cartesian coordinates $x^i_{\rm Cart} = (x,y,z)$, but we need them in the curvilinear coordinate system $x^i_{\rm rfm}= ({\rm xx0},{\rm xx1},{\rm xx2})$ set by the "reference_metric::CoordSystem" variable. Empirically speaking, it is far easier to write $(x({\rm xx0},{\rm xx1},{\rm xx2}),y({\rm xx0},{\rm xx1},{\rm xx2}),z({\rm xx0},{\rm xx1},{\rm xx2}))$ than the inverse, so we will compute the Jacobian matrix

$$
{\rm Jac\_dUCart\_dDrfmUD[i][j]} = \frac{\partial x^i_{\rm Cart}}{\partial x^j_{\rm rfm}},
$$

via exact differentiation (courtesy SymPy), and the inverse Jacobian
$$
{\rm Jac\_dUrfm\_dDCartUD[i][j]} = \frac{\partial x^i_{\rm rfm}}{\partial x^j_{\rm Cart}},
$$

using NRPy+'s ${\rm generic\_matrix\_inverter3x3()}$ function. In terms of these, we have

\begin{align}
\bar{\gamma}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{\gamma}^{\rm Cart}_{\ell m}\\
\bar{A}^{\rm rfm}_{ij} &= 
\frac{\partial x^\ell_{\rm Cart}}{\partial x^i_{\rm rfm}}
\frac{\partial x^m_{\rm Cart}}{\partial x^j_{\rm rfm}} \bar{A}^{\rm Cart}_{\ell m}\\
\bar{\Lambda}^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \bar{\Lambda}^\ell_{\rm Cart} \\
\beta^i_{\rm rfm} &= \frac{\partial x^i_{\rm rfm}}{\partial x^\ell_{\rm Cart}} \beta^\ell_{\rm Cart}
\end{align}

In [8]:
# Step 3: Convert BSSN tensors to basis set by CoordSystem variable above.
Jac_dUCart_dDrfmUD = ixp.zerorank2()
for i in range(DIM):
    for j in range(DIM):
        Jac_dUCart_dDrfmUD[i][j] = sp.diff(rfm.xxCart[i],rfm.xx[j])

Jac_dUrfm_dDCartUD, dummyDET = ixp.generic_matrix_inverter3x3(Jac_dUCart_dDrfmUD)

gammabarDD = ixp.zerorank2()
AbarDD     = ixp.zerorank2()
betaU   = ixp.zerorank1()
LambdaU = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaU[i]   += Jac_dUrfm_dDCartUD[i][j]*betaCartU[j]
        LambdaU[i] += Jac_dUrfm_dDCartUD[i][j]*LambdaCartU[j]
        for k in range(3):
            for l in range(3):
                gammabarDD[i][j] += Jac_dUrfm_dDCartUD[k][i]*Jac_dUrfm_dDCartUD[l][j]*gammaCartbarDD[k][l]
                AbarDD[i][j]     += Jac_dUrfm_dDCartUD[k][i]*Jac_dUrfm_dDCartUD[l][j]*ACartbarDD[k][l]

# cfCart = sp.sympify
# if par.parval_from_str("BSSN_RHSs::ConformalFactor") == "phi":